In [1]:
import random
import numpy as np
import pandas as pd
# Visualisation imports
import matplotlib.pyplot as plt
# Scikit learn for preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Keras Imports - CNN
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Conv1D, MaxPool2D, Flatten, Dropout, Convolution2D, Activation
from keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.linear_model import LogisticRegression
from __future__ import division

Using TensorFlow backend.


In [2]:
data_path = "/Users/MeryemMhamdi/Desktop/Lauzhack_creditSuisse-master/data/"

In [3]:
train = pd.read_csv(data_path+'train.csv', header=0, sep=",", index_col=0, parse_dates=True)

In [4]:
y_train = list(train['suspicious'])
feature_cols = ['category', 'turnover', 'transaction_count', 'io_ratio', 'age', 'nationality', 'is_pep', 'inactive_days_average', 'inactive_days_max', 'n_of_accounts']
X_train = train.loc[:, feature_cols].values

In [5]:
test = pd.read_csv(data_path+'test.csv', header=0, sep=",", index_col=0, parse_dates=True)

In [6]:
X_test = test.loc[:, feature_cols].values

In [7]:
# Encode the categories
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_train = to_categorical(y_train, 2)

In [8]:
im_shape = (X_train.shape[1], 1 , 1)
x_train = X_train.reshape(X_train.shape[0], *im_shape) # Python TIP :the * operator unpacks the tuple
x_test = X_test.reshape(X_test.shape[0], *im_shape)

In [9]:
n_0 = 0
n_1 = 0
for y in list(train['suspicious']):
    if y==0: 
        n_0 +=1
    else:
        n_1 += 1
        
weight_0 = 1/(n_0/len(y_train))
weight_1 = 1/(n_1/len(y_train))
counts = {0: weight_0/ (weight_0 + weight_1), 1 : (weight_0+weight_1)}
weights = []
for y in list(train['suspicious']):
    weights.append(counts[y])

In [10]:
cnn = Sequential()
kernelSize = (3, 3)
ip_activation = 'relu'
#ip_conv_0 = Conv2D(filters=32, kernel_size=kernelSize, input_shape=im_shape, activation=ip_activation)
ip_conv_0 = Convolution2D(32, 3, 3, border_mode='same', input_shape=(10,1,1))
cnn.add(ip_conv_0)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", input_shape=(10, 1, 1))`
  """


In [11]:
# Add the next Convolutional+Activation layer
ip_conv_0_1 = Convolution2D(64, 3, 3, border_mode='same', input_shape=(10,1,1), activation= ip_activation)
cnn.add(ip_conv_0_1)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu", input_shape=(10, 1, 1))`
  


In [12]:
# Add the Pooling layer
pool_0 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="same")
cnn.add(pool_0)

In [13]:
drop_layer_0 = Dropout(0.2)
cnn.add(drop_layer_0)

In [14]:
flat_layer_0 = Flatten()
cnn.add(Flatten())

In [232]:
# Now add the Dense layers
#h_dense_0 = Dense(units=128, activation=ip_activation, kernel_initializer='uniform')
#cnn.add(h_dense_0)
# Let's add one more before proceeding to the output layer
#h_dense_1 = Dense(units=64, activation=ip_activation, kernel_initializer='uniform')
#cnn.add(h_dense_1)

In [18]:
from itertools import product
def w_categorical_crossentropy(y_true, y_pred, weights):
    nb_cl = len(weights)
    final_mask = K.zeros_like(y_pred[:, 0])
    y_pred_max = K.max(y_pred, axis=1)
    y_pred_max = K.reshape(y_pred_max, (K.shape(y_pred)[0], 1))
    y_pred_max_mat = K.equal(y_pred, y_pred_max)
    for c_p, c_t in product(range(nb_cl), range(nb_cl)):
        final_mask += (weights[c_t, c_p] * y_pred_max_mat[:, c_p] * y_true[:, c_t])
    return K.categorical_crossentropy(y_pred, y_true) * final_mask

ncce = partial(w_categorical_crossentropy, weights=np.array(weights))

In [16]:
op_activation = 'softmax'
output_layer = Dense(units=2, activation=op_activation, kernel_initializer='uniform')
cnn.add(output_layer)

In [17]:
from sklearn.metrics import precision_score
import keras.backend as K
from functools import partial

opt = 'adam'
loss = 'categorical_crossentropy'

""""
def custom_loss_4(y_true, y_pred, weights):
    return K.mean(K.abs(y_true - y_pred) * weights)

cl4 = partial(custom_loss_4, weights=weights)

def my_metric(y_true, y_pred):
     return precision_score(y_true, y_pred, average='macro')
     
"""

# Compile the classifier using the configuration we want
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
cnn.compile(optimizer=sgd, loss=loss, metrics=['accuracy'])#, my_metric])

In [19]:
print(cnn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 10, 1, 32)         320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 1, 64)         18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 1, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 1, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 320)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 642       
Total params: 19,458
Trainable params: 19,458
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
history = cnn.fit(x_train[:900000], y_train[:900000], batch_size=32, 
                  epochs=2, validation_data=(x_train[900000:], y_train[900000:]), shuffle=True)

Train on 900000 samples, validate on 100000 samples
Epoch 1/2
900000/900000 [==============================] - 107s 119us/step - loss: 0.1461 - acc: 0.9909 - val_loss: 0.1494 - val_acc: 0.9907
Epoch 2/2
900000/900000 [==============================] - 115s 128us/step - loss: 0.1461 - acc: 0.9909 - val_loss: 0.1494 - val_acc: 0.9907


In [31]:
predicts = cnn.predict(x_test)

In [24]:
scores = cnn.evaluate(x_train[:900000], y_train[:900000], verbose=0)
print("Train Accuracy: %.2f%%" % (scores[1]*100))
scores = cnn.evaluate(x_train[900000:], y_train[900000:], verbose=0)
print("Val Accuracy: %.2f%%" % (scores[1]*100))

Train Accuracy: 99.09%
Val Accuracy: 99.07%


## Ranking of Customers based on their suspicious probabilities:

In [32]:
predicts_suspicion = []
for pred in list(predicts):
    predicts_suspicion.append(pred[1])

In [33]:
test['prob'] = predicts_suspicion

In [34]:
df=test.sort_values(by=['prob'], ascending=False)
df['prob']

customer
91000000    0.0
91666657    0.0
91666659    0.0
91666660    0.0
91666661    0.0
91666662    0.0
91666663    0.0
91666664    0.0
91666665    0.0
91666666    0.0
91666667    0.0
91666668    0.0
91666669    0.0
91666670    0.0
91666671    0.0
91666672    0.0
91666673    0.0
91666674    0.0
91666675    0.0
91666676    0.0
91666677    0.0
91666678    0.0
91666679    0.0
91666680    0.0
91666681    0.0
91666682    0.0
91666683    0.0
91666684    0.0
91666685    0.0
91666658    0.0
           ... 
91333343    0.0
91333342    0.0
91333327    0.0
91333315    0.0
91333316    0.0
91333317    0.0
91333318    0.0
91333319    0.0
91333320    0.0
91333321    0.0
91333322    0.0
91333323    0.0
91333324    0.0
91333325    0.0
91333326    0.0
91333328    0.0
91333341    0.0
91333329    0.0
91333330    0.0
91333331    0.0
91333332    0.0
91333333    0.0
91333334    0.0
91333335    0.0
91333336    0.0
91333337    0.0
91333338    0.0
91333339    0.0
91333340    0.0
91999999    0.0
Name: prob, Len

In [138]:
list(df.index)[:1000]

[91083763,
 91356003,
 91704541,
 91324653,
 91635336,
 91680020,
 91192126,
 91481823,
 91476488,
 91427191,
 91759600,
 91113436,
 91434081,
 91319999,
 91120370,
 91781683,
 91140635,
 91497750,
 91078140,
 91271802,
 91089148,
 91296482,
 91048782,
 91373874,
 91920213,
 91079847,
 91146065,
 91022383,
 91598674,
 91149174,
 91395974,
 91107702,
 91892956,
 91987071,
 91079800,
 91371225,
 91557638,
 91658583,
 91837169,
 91142672,
 91958426,
 91780089,
 91398347,
 91170971,
 91848025,
 91214952,
 91785285,
 91682653,
 91323822,
 91173647,
 91798273,
 91270748,
 91996496,
 91415320,
 91116979,
 91848468,
 91815837,
 91178372,
 91759403,
 91027031,
 91156630,
 91451169,
 91403439,
 91503342,
 91254817,
 91462779,
 91219133,
 91288263,
 91232705,
 91341499,
 91661212,
 91582207,
 91486365,
 91796997,
 91191369,
 91122358,
 91994875,
 91864368,
 91455453,
 91544067,
 91765943,
 91180886,
 91888578,
 91354765,
 91062864,
 91437399,
 91015332,
 91401249,
 91506258,
 91856832,
 91848841,

In [139]:
customers = list(df.index)[:1000]
df_cnn = pd.DataFrame()
df_cnn['customers'] = customers

In [141]:
df_cnn.to_csv('/Users/MeryemMhamdi/Desktop/Lauzhack_creditSuisse/results/CNN2-Correctformat.csv', header=['customer'], index=False)

## Cost Function:

S = I * N + F * P * M